## Problem set: Week 7 (Resting fMRI)
In this problem set you will load the correlation data from one of the Midnight Scan Club subjects (which has already been extracted using the Glasser MMP parcellation) and perform several analyses to characterize the network.

As before, skeletal code is provided - please fill in any areas where you see ...


In [1]:
import os,pickle,sys
import numpy,pandas
import nilearn.datasets
import nilearn.plotting
import matplotlib.pyplot as plt
import scipy.stats
import networkx as nx
import sklearn.metrics
import bct
from collections import Counter

from brainnetworks.r2z import r_to_z,z_to_r
%matplotlib inline

datadir = nilearn.datasets.get_data_dirs()[0]
if not os.path.exists(datadir):
    os.mkdir(datadir)
    
atlasdir='/home/vagrant/brain-networks-course/data/HCP-MMP1'

labelfile=os.path.join(atlasdir,'MMP_yeo2011_networks.csv')
labeldata=pandas.read_csv(labelfile)


In [2]:
# first load the data

sub=1
corrtype='gsr'  # use data with global signal regression
scrubtype='full' # don't use scrubbing


subdir=os.path.join(datadir,'MSC/ds000224/derivatives/fmriprep/sub-MSC%02d/'%sub)

corrs=pickle.load(open(os.path.join(subdir,'sub-MSC%02d_task-rest_corrmtx.pkl'%sub),'rb'))



Now compute the mean correlation matrix across sesssions, using the r-to-z transform to first convert them to Z scores and then convert back to r values after averaging. The correlation data are stored in a dictionary, with the following key structure:

> ```corrs[session num][corrtype:{'gsr','nogsr'}][scrubtype:{'scrubbed','full'}]```

We will use corrtype and scrubtype as specified above

In [3]:
corrsum=numpy.zeros(corrs[1][corrtype][scrubtype].shape)
for s in corrs:
    sesscor=corrs[s][corrtype][scrubtype]
    corrsum+=r_to_z(sesscor)
    
meancorr=z_to_r(corrsum/len(corrs))
    



/home/vagrant/anaconda3/envs/py3/lib/python3.6/site-packages/brainnetworks/r2z.py:9: RuntimeWarning: divide by zero encountered in true_divide
  z=0.5*numpy.log((1.0+r)/(1.0-r))


**Problem 1**: Create a binary adjacency matrix with a density of 5%, and use this to create a NetworkX graph.  Be sure to do the following:

- exclude the diagonal when computing the cutoff 
- zero out the diagonal before creating the graph
- extract the giant component from the graph (calling the resulting variable ```Gc```)
- print the number of nodes in the giant component

In [15]:
# determine cutoff for 5% density of the correlation matrix
# using just the upper triangle of the matrix
thresh=95  # in percent
cutoff=scipy.stats.scoreatpercentile(meancorr[numpy.triu_indices_from(meancorr,1)],
                                         100-(100*0.05))

#create symmetric binary adjacency matrix
# be sure to convert to integer
adjmtx=(meancorr>cutoff).astype('int')

# zero out the diagonal in the adjmtx
adjmtx[numpy.diag_indices_from(adjmtx)]=0

# Create numpy graph
G=nx.from_numpy_array(adjmtx)

# create graph for giant component
# first get all component subgraphs
comps=[i for i in nx.connected_component_subgraphs(G)]
# then take the largest
Gc=comps[0]

print('Giant component includes %d out of %d total nodes'%(len(Gc.nodes),len(G.nodes)))

#Gc adjacency matrix
adjmtxGc=nx.adjacency_matrix(Gc, nodelist=None, weight='weight')


# grab the label data for only the nodes in the giant component
labeldata_Gc=labeldata.loc[list(Gc.nodes)]
# add degree values to labeldata frame
labeldata_Gc['degree']=[Gc.degree[i] for i in labeldata_Gc.index]




Giant component includes 356 out of 360 total nodes


/home/vagrant/anaconda3/envs/py3/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


**Problem 3**: Perform community detection on the graph, using the Louvain algorithm for undirected binary graphs as implemented in the bct python package, and compute their overlap with the Yeo 7 network parcellation



In [16]:
# compute modularity using bct
mod_binary=bct.modularity_louvain_und(adjmtxGc)

print('modularity:',mod_binary[1])
print('Multilevel modularity optimization identifed %d communities'%len(numpy.unique(mod_binary[0])))

# compute adjusted rand score using method from sklearn.metrics
ari=sklearn.metrics.adjusted_rand_score(mod_binary[0],labeldata_Gc['Yeo7'])
print('Adjusted Rand index compared to Yeo 7 networks: %0.3f'%ari)



TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

**Problem 4**: Estimate the normalized rich club coefficient for this network and plot the coefficients across the range of degree values.  Find the smallest degree value  for which the rich club coefficient is greater than 2, which we will use to define the rich club nodes.

In [17]:
# embed computation of rcc within a try/catch since it fails
# pretty regularly with a ZeroDivisionError
good_rcc=False
while not good_rcc:
    try:
        rcc = nx.rich_club_coefficient(Gc,normalized=True,Q=10)
        good_rcc=True
    except ZeroDivisionError:
        print('error, retrying')
        
# put into a data frame
rccdata=pandas.DataFrame([(i,rcc[i]) for i in rcc.keys()],
                         columns=['degree','rcc'])

# find the degree cutoff for rcc >= 2
degree_cutoff=numpy.min(rccdata[rccdata['rcc'] >= 2]['degree'])
print('degree_cutoff:',degree_cutoff) 

# compute the size of the rich club
rc_size=numpy.sum(rccdata['degree'] >= degree_cutoff)
print(rc_size,'nodes in rich club')

degree_cutoff: 35
13 nodes in rich club


**Problem 5:** For each of the Yeo7 networks, determine how many rich club members fall within that network.

In [9]:
# first create a data frame containing label data just for rcc members

labeldata_rcc=labeldata_Gc[labeldata_Gc['degree'] >= degree_cutoff]

# use collections.Counter to generate a list of the counts of members in each
# Yeo7 network
c=Counter(labeldata_rcc['YeoDesc7'])
print(c)

Counter({'Visual': 21, 'VentralAttention': 13, 'DorsalAttention': 6, 'Default': 2, 'Somatomotor': 1})


**Problem 6:** First, compute the node betweenness centrality and edge betweeness centrality for the giant component network.  

In [10]:
# compute edge betweenness centrality
ebc=nx.edge_betweenness_centrality(Gc)

# compute node betweenness centrality
bc=nx.betweenness_centrality(Gc)

Then, compute the mean betweenness centrality for edges separated by whether they include 0, 1, or 2 members of the rich club, and print out the mean values for each.

In [11]:
Gc_rcc=Gc.copy()
deg = dict(Gc_rcc.degree()) 
to_remove = [n for n in deg if deg[n] < degree_cutoff]
Gc_rcc.remove_nodes_from(to_remove)
richclubnodes=list(Gc_rcc.nodes)

edges=ebc.keys()
include_no_members=[]
include_one_member=[]
include_two_members=[]
for edge in edges: 
    if edge[0] not in richclubnodes and edge[1] not in richclubnodes:
        include_no_members.append(ebc[edge])
    if (edge[0] in richclubnodes and edge[1] not in richclubnodes) or (edge[1] in richclubnodes and edge[0] not in richclubnodes):
        include_one_member.append(ebc[edge])
    if edge[0] in richclubnodes and edge[1] in richclubnodes:
        include_two_members.append(ebc[edge])

no_member_mean=numpy.mean(include_no_members)
one_member_mean=numpy.mean(include_one_member)
two_member_mean=numpy.mean(include_two_members)
print('no members mean:', no_member_mean)
print('one members mean:', one_member_mean)
print('two members mean:', two_member_mean)

no members mean: 0.001303639459318467
one members mean: 0.0010032096606917421
two members mean: 0.0004747857984713991


Now compute the mean betweenness centrality for nodes, separated by whether the nodes are members of the rich club or not, and print the values for each.

In [12]:
inrichclubnodes=[]
notinrichclubnodes=[]
for node in list(Gc.nodes):
    if node in richclubnodes:
        inrichclubnodes.append(bc[node])
    if node not in richclubnodes:
        notinrichclubnodes.append(bc[node])
in_mean=numpy.mean(inrichclubnodes)
notin_mean=numpy.mean(notinrichclubnodes)
print('in:', in_mean)
print('not in:', notin_mean)

in: 0.012539901048146873
not in: 0.006572179058063817


How does centrality of both nodes and edges relate to rich club membership?  Please explain (insert your answer in the following cell).

Edge centrality distinguishes between nodes in the rich club membership that are connected to edges that are most commonly taken in shortest paths, thus placing the connected nodes in the rich club. Node centrality distinguishes between nodes that are most passed through in paths, thus the ones that are most passed through are in the rich club. 